IMPORT libraries and dataset

In [41]:
import pandas as pd
import numpy as np
import ast
import tensorflow as tf
from tensorflow.keras import layers, models
import os
import gc

In [2]:
gpus=tf.config.list_physical_devices('GPU')
for gpu in gpus: 
    tf.config.experimental.set_memory_growth(gpu, True)


I0000 00:00:1728582338.723903       9 cuda_executor.cc:1001] could not open file to read NUMA node: /sys/bus/pci/devices/0000:01:00.0/numa_node
Your kernel may have been built without NUMA support.
I0000 00:00:1728582338.810788       9 cuda_executor.cc:1001] could not open file to read NUMA node: /sys/bus/pci/devices/0000:01:00.0/numa_node
Your kernel may have been built without NUMA support.
I0000 00:00:1728582338.810894       9 cuda_executor.cc:1001] could not open file to read NUMA node: /sys/bus/pci/devices/0000:01:00.0/numa_node
Your kernel may have been built without NUMA support.


In [4]:
dataset_2=pd.read_csv('./AI-Module/Resources/Datasets/how2sign2.csv')
dataset = pd.read_csv("./AI-Module/Resources/Datasets/how2sign.csv")

In [5]:
puntos_folder = "/app/AI-Module/Resources/Datasets/Points"
files = [puntos_folder + "/" + file for file in os.listdir(puntos_folder)]
len(files)

500

In [6]:
len(dataset)

500

In [7]:
def load_points(files):
    for file in files:
        item_with_index = np.load(file, allow_pickle=True)
        # Convertir los arrays a float32 para reducir memoria
        item = item_with_index[0].astype(np.float32)
        index = item_with_index[1]
        yield item, index

puntos_list = []
ids_list = []

for item, index in load_points(files):
    puntos_list.append(item)
    ids_list.append(index)

df_puntos = pd.DataFrame({
    'points': puntos_list,
    'id': ids_list
})
df_puntos = df_puntos.sort_values(["id"]).reset_index(drop=True)

In [8]:
dataset = dataset.merge(df_puntos, on="id", how="inner")

In [9]:
dataset.head()

,points_x,translation,id,len_keyframes,points_y
0,"[[-1.0, -1.0, -1.0, 0.0, -1.0, -1.0, -1.0, 0.0...",If you like to find more about my services you...,448,817,"[[-1.0, -1.0, -1.0, 0.0, -1.0, -1.0, -1.0, 0.0..."
1,"[[-1.0, -1.0, -1.0, 0.0, -1.0, -1.0, -1.0, 0.0...",Hello welcome my name is Julio Nutt and I am a...,210,401,"[[-1.0, -1.0, -1.0, 0.0, -1.0, -1.0, -1.0, 0.0..."
2,"[[1.0, 0.0589, 1.0, 1.0, 0.7364, 0.0, 0.6998, ...","All right, the drink we're about to make is ca...",228,308,"[[1.0, 0.0589, 1.0, 1.0, 0.7364, 0.0, 0.6998, ..."
3,"[[1.0, 0.0515, 1.0, 1.0, 0.7142, 0.0, 0.6933, ...","Okay, another thing that for me is important. ...",187,367,"[[1.0, 0.0515, 1.0, 1.0, 0.7142, 0.0, 0.6933, ..."
4,"[[-1.0, -1.0, -1.0, 0.0, -1.0, -1.0, -1.0, 0.0...","All right, we're ready to start our fire. What...",99,453,"[[-1.0, -1.0, -1.0, 0.0, -1.0, -1.0, -1.0, 0.0..."


In [10]:
dataset["points_y"].iloc[0]

array([[-1.    , -1.    , -1.    , ...,  0.3154,  0.4075,  1.    ],
       [-1.    , -1.    , -1.    , ...,  0.2917,  0.4159,  1.    ],
       [ 1.    ,  0.0773,  1.    , ...,  0.3163,  0.4409,  1.    ],
       ...,
       [ 1.    ,  0.1223,  0.9016, ...,  0.3382,  0.3776,  1.    ],
       [ 1.    ,  0.1332,  0.8529, ...,  0.3337,  0.364 ,  1.    ],
       [ 1.    ,  0.1269,  0.7338, ...,  0.338 ,  0.3777,  1.    ]],
      dtype=float32)

In [11]:
dataset["points"]=dataset["points_y"]

In [ ]:
puntos_folder = "/app/AI-Module/Resources/Datasets/Points2"
files = [puntos_folder + "/" + file for file in os.listdir(puntos_folder)]
len(files)

In [ ]:
def load_points(files):
    for file in files:
        item_with_index = np.load(file, allow_pickle=True)
        # Convertir los arrays a float32 para reducir memoria
        item = item_with_index[0].astype(np.float32)
        index = item_with_index[1]
        yield item, index

puntos_list = []
ids_list = []

for item, index in load_points(files):
    puntos_list.append(item)
    ids_list.append(index)

df_puntos = pd.DataFrame({
    'points': puntos_list,
    'id': ids_list
})
df_puntos = df_puntos.sort_values(["id"]).reset_index(drop=True)

In [ ]:
dataset_2 = dataset_2.merge(df_puntos, on="id", how="inner")

Cambio el formato de points para que sea una lista de 48x48

In [11]:
# import sys
# # Agrega la ruta donde está ubicado Points2VecClass.py
# sys.path.append('/app/AI-Module/Modules')
# from Points2VecClass import Point2Vec
# def pointsToCnnInputForm(points):
#     p2v = Point2Vec(4)
#     return p2v.CNNMatrix(points)

In [12]:
# def to_cnn_input_form(serie):
#     for index, element in enumerate(serie):
#         element= pointsToCnnInputForm(element)
#         serie.at[index]=element
#     return serie
# dataset['points']=to_cnn_input_form(dataset['points'])

Tokenizo las traducciones para que sean compatibles con el modelo

In [ ]:
import json
from tensorflow.keras.preprocessing.text import tokenizer_from_json

# Cargar el archivo JSON
with open('./tokenizer_entero.json') as json_file:
    tokenizer_json = json.load(json_file)

# Reconstruir el tokenizer desde el archivo JSON
tokenizer = tokenizer_from_json(tokenizer_json)

In [12]:
# from tensorflow.keras.preprocessing.text import Tokenizer

# # Crear el tokenizador
# tokenizer = Tokenizer()

# # Ajustar el tokenizador al texto de las traducciones
# tokenizer.fit_on_texts(dataset['translation'])

# # Convertir las oraciones en secuencias de enteros
dataset['translation'] = tokenizer.texts_to_sequences(dataset['translation'])
dataset_2['translation']=tokenizer.texts_to_sequence(dataset['translation'])

In [13]:
from sklearn.model_selection import train_test_split


# Divide en train y test
points_train_val, points_test, translation_train_val, translation_test = train_test_split(
    dataset[["points", "len_keyframes"]], dataset["translation"], test_size=0.2, random_state=42
)
print(f"Train_val size: {len(points_train_val)}, Test size: {len(points_test)}")
points_train, points_val, translation_train, translation_val=train_test_split(
    points_train_val,translation_train_val, test_size=0.25,random_state=42
)
print(f"Train size: {len(points_train)}, Val size: {len(points_val)}")
train = {
    "points": points_train["points"],
    "len_keyframes": points_train["len_keyframes"],
    "translation_sequence": translation_train
}

val = {
    "points": points_val["points"],
    "len_keyframes": points_val["len_keyframes"],
    "translation_sequence": translation_val
}

test = {
    "points": points_test["points"],
    "len_keyframes": points_test["len_keyframes"],
    "translation_sequence": translation_test
}


Train_val size: 400, Test size: 100
Train size: 300, Val size: 100


In [ ]:
points_train, points_val, translation_train, translation_val = train_test_split(
    dataset_2[["points", "len_keyframes"]], dataset_2["translation"], test_size=0.15, random_state=42
)
print(f"Train_val size: {len(points_train_val)}, Test size: {len(points_test)}")

train2 = {
    "points": points_train["points"],
    "len_keyframes": points_train["len_keyframes"],
    "translation_sequence": translation_train
}

val2 = {
    "points": points_val["points"],
    "len_keyframes": points_val["len_keyframes"],
    "translation_sequence": translation_val
}

In [15]:


globals().pop('dataset', None)
globals().pop('puntos_folder', None)
globals().pop('puntos_list', None)
globals().pop('df_puntos', None)
gc.collect()

0

In [14]:
len(train['translation_sequence'])

300

Creo datasets de tensorflow

In [15]:
def create_tf_dataset(df):
    def generator():
        for i in range(len(df['points'])):
            points = df['points'].values[i]
            sequence = df['translation_sequence'].values[i]
            yield points, sequence

    return tf.data.Dataset.from_generator(
        generator,
        output_signature=(
            # tf.TensorSpec(shape=(None, 48, 48), dtype=tf.float32),
            tf.TensorSpec(shape=(None, 2172), dtype=tf.float32),
            tf.TensorSpec(shape=(None), dtype=tf.int32)
        )
)


In [16]:
test_dataset = create_tf_dataset(test)
val_dataset=create_tf_dataset(val)
train_dataset= create_tf_dataset(train)
train_dataset_2=create_tf_dataset(train2)
val_dataset_2=create_tf_dataset(val2)

I0000 00:00:1728582509.321837       9 cuda_executor.cc:1001] could not open file to read NUMA node: /sys/bus/pci/devices/0000:01:00.0/numa_node
Your kernel may have been built without NUMA support.
I0000 00:00:1728582509.322024       9 cuda_executor.cc:1001] could not open file to read NUMA node: /sys/bus/pci/devices/0000:01:00.0/numa_node
Your kernel may have been built without NUMA support.
I0000 00:00:1728582509.322070       9 cuda_executor.cc:1001] could not open file to read NUMA node: /sys/bus/pci/devices/0000:01:00.0/numa_node
Your kernel may have been built without NUMA support.
I0000 00:00:1728582515.248880       9 cuda_executor.cc:1001] could not open file to read NUMA node: /sys/bus/pci/devices/0000:01:00.0/numa_node
Your kernel may have been built without NUMA support.
I0000 00:00:1728582515.248992       9 cuda_executor.cc:1001] could not open file to read NUMA node: /sys/bus/pci/devices/0000:01:00.0/numa_node
Your kernel may have been built without NUMA support.
2024-10-10

In [ ]:
globals().pop('test', None)
globals().pop('train', None)
globals().pop('val', None)
gc.collect()

In [17]:
train_dataset=train_dataset.shuffle(buffer_size=10000)
val_dataset=val_dataset.shuffle(buffer_size=10000)
test_dataset=test_dataset.shuffle(buffer_size=10000)

In [ ]:
train_dataset.save('./train')
val_dataset.save('./val')
test_dataset.save('./test')
train_dataset_2.save('./train2')
val_dataset_2.save('./val2')

In [18]:
count = train_dataset.reduce(0, lambda x, _: x + 1)
count.numpy()

300

In [19]:
print(val_dataset.element_spec)

(TensorSpec(shape=(None, 2172), dtype=tf.float32, name=None), TensorSpec(shape=<unknown>, dtype=tf.int32, name=None))


In [29]:
for element in train_dataset.take(1):
    print(element)

(<tf.Tensor: shape=(268, 2172), dtype=float32, numpy=
array([[1.    , 0.097 , 0.6709, ..., 0.3187, 0.3373, 1.    ],
       [1.    , 0.1203, 0.7383, ..., 0.3126, 0.3504, 1.    ],
       [1.    , 0.104 , 0.7192, ..., 0.3115, 0.3603, 1.    ],
       ...,
       [1.    , 0.6054, 1.    , ..., 0.3259, 0.3816, 1.    ],
       [1.    , 0.5996, 1.    , ..., 0.3262, 0.3763, 1.    ],
       [1.    , 0.5957, 1.    , ..., 0.324 , 0.3744, 1.    ]],
      dtype=float32)>, <tf.Tensor: shape=(200,), dtype=int32, numpy=
array([  14,   41,   19,   59, 3203,   92,    8,    4,   86,    2,  104,
         11,  572,   53,   33,  140, 1911,   27,    9,    7, 1337, 6602,
         38,   22, 1159,   41,   19,  645,    6, 6603,  101,   10,  573,
          7,   50,   12,    2, 6604,  234,  111,  329,    4,  376,   31,
          5,  248,  210,    9,  542,  233,   32,    8,   22,   95,  904,
          6,   91,   14,   23,  929,    6,    8,   45,   82,    8,   10,
        573,    7,   50,   11,    5, 6605,  527,    6,

2024-10-10 14:22:52.542883: I tensorflow/core/framework/local_rendezvous.cc:404] Local rendezvous is aborting with status: OUT_OF_RANGE: End of sequence


In [41]:
def print_lenForBucketing(ds,limites):

    # Crear un DataFrame vacío para almacenar los resultados
    resultados = []

    # Recorrer los pares consecutivos de la lista de límites
    for lower, upper in zip(limites[:-1], limites[1:]):
        # Contar cuántos valores están entre 'lower' y 'upper'
        count = ((ds['len_keyframes'] > lower) & (ds['len_keyframes'] <= upper)).sum()
        
        # Guardar el resultado en una lista
        resultados.append((lower, upper, count))

    # Mostrar los resultados
    for lower, upper, count in resultados:
        print(f"Valores entre {lower} y {upper}: {count}")


limites = [0,300,500,700, 1000]
print(f'Dataset: test')
print_lenForBucketing(test,limites)
print(f'Dataset: train')
print_lenForBucketing(train,limites)
print(f'Dataset: val')

print_lenForBucketing(val,limites)

Dataset: test
Valores entre 0 y 300: 17
Valores entre 300 y 500: 52
Valores entre 500 y 700: 21
Valores entre 700 y 1000: 7
Dataset: train
Valores entre 0 y 300: 57
Valores entre 300 y 500: 154
Valores entre 500 y 700: 55
Valores entre 700 y 1000: 24
Dataset: val
Valores entre 0 y 300: 18
Valores entre 300 y 500: 51
Valores entre 500 y 700: 20
Valores entre 700 y 1000: 11


In [25]:
train_dataset

<_FlatMapDataset element_spec=(TensorSpec(shape=(None, 48, 48), dtype=tf.float32, name=None), TensorSpec(shape=<unknown>, dtype=tf.int32, name=None))>

In [20]:
def length_fn(points, translations):
    # La longitud será la cantidad de keyframes (primera dimensión de los puntos)
    longitud=tf.shape(points)[0]
    return longitud


# limites = [300,500,700,1100]
    
# bucket_batch_sizes = [32, 32, 16, 8,8]

def bucketed_dataset(dataset):
    limites = [300,500,700,1100]
    
    bucket_batch_sizes = [8, 8, 8, 4, 4]
    return dataset.bucket_by_sequence_length(
            element_length_func=length_fn,       
            bucket_boundaries=limites, # Límites de los buckets
            bucket_batch_sizes=bucket_batch_sizes,  # Tamaños de batch para cada bucket
            padded_shapes=(
                # [None,48,48],
                [None, 2172],  
                [None]           
            ),
            padding_values=(
                tf.constant(0, dtype=tf.float32), # Relleno con ceros para los puntos 
                tf.constant(0, dtype=tf.int32) # Relleno con ceros para las traducciones 
            )
    )

# Aplicar bucketing y padding
train_dataset= train_dataset.apply(bucketed_dataset)
val_dataset= val_dataset.apply(bucketed_dataset)
test_dataset= test_dataset.apply(bucketed_dataset)



In [21]:
train_dataset=train_dataset.shuffle(buffer_size=100)
val_dataset=val_dataset.shuffle(buffer_size=100)
test_dataset=test_dataset.shuffle(buffer_size=100)

In [22]:
def count_batches(dataset):
    batch_count = 0
    for _ in dataset:
        batch_count += 1
    return batch_count
print(f"Cantidad de batches en train: {count_batches(train_dataset)}")
print(f"Cantidad de batches en val: {count_batches(val_dataset)}")
print(f"Cantidad de batches en test: {count_batches(test_dataset)}")


2024-10-10 17:49:12.013961: I tensorflow/core/framework/local_rendezvous.cc:404] Local rendezvous is aborting with status: OUT_OF_RANGE: End of sequence


Cantidad de batches en train: 44


2024-10-10 17:49:12.483850: I tensorflow/core/framework/local_rendezvous.cc:404] Local rendezvous is aborting with status: OUT_OF_RANGE: End of sequence


Cantidad de batches en val: 16
Cantidad de batches en test: 16


Función de pérdida

In [33]:
# def ctc_loss(translation, prediction):
    
#     input_length = tf.fill([tf.shape(prediction)[0], 1], tf.shape(prediction)[1])  # Longitud de la secuencia de entrada
#     label_length = tf.math.count_nonzero(translation, axis=-1, keepdims=True)   # Longitud de la secuencia de traducción
#     # Calcular la pérdida CTC usando ctc_batch_cost
#     loss = tf.keras.backend.ctc_batch_cost(translation, prediction, input_length, label_length)
#     return loss

**Buscar mejores parámetros**

In [ ]:
# import keras_tuner as kt
# num_classes=len(tokenizer.word_index) + 1

# # Definir el modelo para la búsqueda de hiperparámetros
# def build_model(hp):

#     def createCNNkt():

#         model= models.Sequential()
#         # Añadir capas CNN dinámicamente según el número de capas que se elija (1 a 5 capas CNN)
#         for i in range(hp.Int('num_cnn_layers', 2, 5)):  # De 1 a 5 capas CNN
#             model.add(layers.Conv2D(
#                 filters=hp.Int(f'conv_{i+1}_filters', min_value=64, max_value=256, step=32),
#                 kernel_size=(3, 3),
#                 activation='relu',
#                 input_shape=(48, 48, 1) if i == 0 else None))
#             model.add(layers.MaxPooling2D((2, 2)))

#         model.add(layers.Flatten())
#         return model

#     def createLSTMkt():

#         model= models.Sequential()
#         # Añadir capas LSTM dinámicamente según el número de capas que se elija (1 a 5 capas LSTM)
#         for i in range(hp.Int('num_lstm_layers', 2, 5)):  # De 1 a 5 capas LSTM
#             model.add(layers.LSTM(
#                 units=hp.Int(f'lstm_{i+1}_units', min_value=64, max_value=256, step=32),
#                 return_sequences=True))

#             # Añadir Dropout para cada capa LSTM
#             model.add(layers.Dropout(hp.Float(f'dropout_{i+1}', min_value=0.2, max_value=0.4, step=0.1)))
#         return model

#     cnn = createCNNkt()

#     video_input = layers.Input(shape=(None, 48, 48, 1))

#     # Aplicar CNN a cada frame usando TimeDistributed
#     cnn_features = layers.TimeDistributed(cnn)(video_input)

#     lstm=createLSTMkt()
#     lstm_out= lstm(cnn_features)
#     # Capa final de salida
#     output = layers.Dense(num_classes, activation='linear')(lstm_out)

#     # Compilar el modelo
#     model.compile(optimizer='adam', loss=ctc_loss)

#     model = models.Model(inputs=video_input, outputs=output)

#     return model

# # Definir la búsqueda de hiperparámetros con Keras Tuner
# tuner = kt.Hyperband(
#     build_model,
#     objective='val_loss', # Métrica de evaluación
#     max_epochs= 50,
#     factor=3,
#     directory='',
#     project_name='cnn_lstm_hyperparam'
# )

In [ ]:
# # Realizar la búsqueda de hiperparámetros
# tuner.search(train_dataset, validation_data=val_dataset)

# best_model=tuner.get_best_model(1)[0]
# # Obtener los mejores hiperparámetros encontrados
# best_hyperparams = tuner.get_best_hyperparameters(num_trials=1)[0]

In [ ]:
# print(best_hyperparams)

**Crear Modelo**

CNN

In [25]:
# def create_cnn():
#     model = models.Sequential()
#     model.add(layers.Input(shape=(48, 48, 1)))
#     model.add(layers.Conv2D(32, (3, 3), activation='relu'))
#     model.add(layers.MaxPooling2D((2, 2)))
#     model.add(layers.Conv2D(64, (3, 3), activation='relu'))
#     model.add(layers.MaxPooling2D((2, 2)))
#     model.add(layers.Flatten())
#     return model

In [23]:
def create_conv1d():
    model = models.Sequential()
    model.add(layers.Input(shape=(2172, 1)))
    model.add(layers.Conv1D(64, kernel_size=5, activation='relu'))
    model.add(layers.MaxPooling1D(pool_size=2))
    model.add(layers.Conv1D(64, kernel_size=3, activation='relu'))
    model.add(layers.MaxPooling1D(pool_size=2))
    model.add(layers.Flatten())
    return model

In [24]:
cnn_example_model = create_conv1d()


LSTM

In [25]:
def create_lstm():
    model=models.Sequential()
    # Primera capa LSTM con return_sequences=True
    model.add(layers.Bidirectional(layers.LSTM(64, return_sequences=True)))
    model.add(layers.Dropout(0.3)) # Añadir Dropout
    
    # Segunda capa LSTM con return_sequences=True
    model.add(layers.LSTM(128, return_sequences=True))
    model.add(layers.Dropout(0.3))

    return model

CNN+LSTM

In [26]:
def create_cnn_lstm_model(num_classes):
    conv1d = create_conv1d()
    
    video_input = layers.Input(shape=(None, 2172, 1)) 
    # masked_input = layers.Masking(mask_value=0.0)(video_input)

    
    # Aplicar CNN a cada frame usando TimeDistributed
    cnn_features = layers.TimeDistributed(conv1d)(video_input)
    
    lstm=create_lstm()
    lstm_out= lstm(cnn_features)
    # Capa final de salida
    output = layers.Dense(num_classes, activation='linear')(lstm_out)
    
    model = models.Model(inputs=video_input, outputs=output)
    return model

Implementación para el entrenamiento

Usar bleu score

In [27]:
%pip install nltk

   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 1.5/1.5 MB 7.8 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 792.8/792.8 kB 6.4 MB/s eta 0:00:00
Note: you may need to restart the kernel to use updated packages.


In [28]:
import nltk
from nltk.translate.bleu_score import sentence_bleu, SmoothingFunction

def calculate_bleu(references, hypotheses):
    smoothing = SmoothingFunction().method4
    return np.mean([sentence_bleu([ref], hyp, smoothing_function=smoothing) for ref, hyp in zip(references, hypotheses)])

In [29]:
num_classes=len(tokenizer.word_index) + 1
model = create_cnn_lstm_model(num_classes)

optimizador = tf.keras.optimizers.Adam(learning_rate=0.01)
model.compile(optimizer=optimizador)
checkpoint_callback = tf.keras.callbacks.ModelCheckpoint(
    filepath='/best_model.keras',
    save_best_only=True,       # Guardar solo si es el mejor modelo hasta ahora
    monitor='val_loss',
    mode='min'  
)

early_stopping = tf.keras.callbacks.EarlyStopping(patience=4, restore_best_weights=True)

In [30]:
!nvidia-smi

Thu Oct 10 17:49:33 2024       
+---------------------------------------------------------------------------------------+
| NVIDIA-SMI 535.157                Driver Version: 538.18       CUDA Version: 12.2     |
|-----------------------------------------+----------------------+----------------------+
| GPU  Name                 Persistence-M | Bus-Id        Disp.A | Volatile Uncorr. ECC |
| Fan  Temp   Perf          Pwr:Usage/Cap |         Memory-Usage | GPU-Util  Compute M. |
|                                         |                      |               MIG M. |
|=========================================+======================+======================|
|   0  NVIDIA RTX 3500 Ada Gene...    On  | 00000000:01:00.0 Off |                  Off |
| N/A   45C    P8              11W /  35W |    567MiB / 12282MiB |      0%      Default |
|                                         |                      |                  N/A |
+-----------------------------------------+----------------------+--

In [31]:
ctc_loss=tf.keras.losses.CTC()

In [32]:
def decode(predictions,greedy):
    predicted_classes = tf.argmax(predictions, axis=-1)
    mask = tf.not_equal(predicted_classes, 0)
    seq_lengths = tf.reduce_sum(tf.cast(mask, tf.int32), axis=-1)
    
    predictions=tf.transpose(predictions,[1,0,2])

    if greedy:
        decoded, _ = tf.nn.ctc_greedy_decoder(inputs=predictions, sequence_length=seq_lengths)
    
    else:
        decoded, _ = tf.nn.ctc_beam_search_decoder(inputs=predictions, sequence_length=seq_lengths)
    decoded_dense = tf.sparse.to_dense(decoded[0], default_value=-1).numpy()
    
    # Extraer las secuencias decodificadas eliminando los -1 (valores de relleno)
    decoded_sequences = []
    for seq in decoded_dense:
        decoded_sequences.append([val for val in seq if val != -1])
    
    return decoded_sequences

In [33]:
print(num_classes)

7664


In [34]:
def trainstep(model, points_batch, translation_batch,step, epoch_loss_avg, epoch_accuracy):
    with tf.GradientTape() as tape:
        predictions = model(points_batch, training=True)
        loss = tf.reduce_mean(ctc_loss(translation_batch, predictions))
        
    # Calcular y aplicar gradientes
    gradients = tape.gradient(loss, model.trainable_variables)
    model.optimizer.apply_gradients(zip(gradients, model.trainable_variables))
    # if step % 50 == 0:
    #     print(f"Step {step}: loss = {loss.numpy()}")
    predictions_decoded=decode(predictions, greedy=True)
    epoch_accuracy.update_state(translation_batch, predictions_decoded)
    epoch_loss_avg.update_state(loss)

In [35]:
def valstep(model, points_batch, translation_batch,val_loss_avg, val_acc_metric, val_bleu_avg):
    # Evaluar en el conjunto de validación al final de cada epoch usando BLEU
   
    predictions = model(points_batch, training=False)

    val_loss=tf.reduce_mean(ctc_loss(translation_batch,predictions))
    val_loss_avg.update_state(val_loss)

    val_predictions = decode(predictions, greedy=False)
    val_hypotheses = [tokenizer.sequences_to_texts([pred]) for pred in val_predictions]


    val_acc_metric.update_state(translation_batch,val_predictions)
    val_references = [tokenizer.sequences_to_texts([ref.numpy()]) for ref in translation_batch]
    val_hypotheses = [tokenizer.sequences_to_texts([pred]) for pred in np.argmax(val_predictions, axis=-1)]
    val_bleu = calculate_bleu(references = val_references, hypotheses = val_hypotheses)
    val_bleu_avg.update_state(val_bleu)
    return val_acc_metric.result(), val_loss_avg.result(), val_bleu_avg.result()

In [48]:
train_dataset.save('./train')
val_dataset.save('./val')
test_dataset.save('./test')

In [37]:
!nvidia-smi

Thu Oct 10 17:49:50 2024       
+---------------------------------------------------------------------------------------+
| NVIDIA-SMI 535.157                Driver Version: 538.18       CUDA Version: 12.2     |
|-----------------------------------------+----------------------+----------------------+
| GPU  Name                 Persistence-M | Bus-Id        Disp.A | Volatile Uncorr. ECC |
| Fan  Temp   Perf          Pwr:Usage/Cap |         Memory-Usage | GPU-Util  Compute M. |
|                                         |                      |               MIG M. |
|=========================================+======================+======================|
|   0  NVIDIA RTX 3500 Ada Gene...    On  | 00000000:01:00.0 Off |                  Off |
| N/A   45C    P8              N/A /  35W |    567MiB / 12282MiB |      0%      Default |
|                                         |                      |                  N/A |
+-----------------------------------------+----------------------+--

In [44]:
train_dataset=tf.data.Dataset.load('/train')
val_dataset=tf.data.Dataset.load('/val')
test_dataset=tf.data.Dataset.load('/test')

In [45]:
!nvidia-smi

Thu Oct 10 17:53:12 2024       
+---------------------------------------------------------------------------------------+
| NVIDIA-SMI 535.157                Driver Version: 538.18       CUDA Version: 12.2     |
|-----------------------------------------+----------------------+----------------------+
| GPU  Name                 Persistence-M | Bus-Id        Disp.A | Volatile Uncorr. ECC |
| Fan  Temp   Perf          Pwr:Usage/Cap |         Memory-Usage | GPU-Util  Compute M. |
|                                         |                      |               MIG M. |
|=========================================+======================+======================|
|   0  NVIDIA RTX 3500 Ada Gene...    On  | 00000000:01:00.0 Off |                  Off |
| N/A   44C    P8               3W /  35W |    567MiB / 12282MiB |      0%      Default |
|                                         |                      |                  N/A |
+-----------------------------------------+----------------------+--

In [65]:
from tqdm import tqdm
points_batch, translation_batch = next(iter(train_dataset))
points_batch.shape, translation_batch.shape

Entrenamiento: 0batch [00:01, ?batch/s]


(TensorShape([8, 297, 2172]), TensorShape([8, 231]))

In [73]:
!nvidia-smi

Thu Oct 10 14:47:57 2024       
+---------------------------------------------------------------------------------------+
| NVIDIA-SMI 535.157                Driver Version: 538.18       CUDA Version: 12.2     |
|-----------------------------------------+----------------------+----------------------+
| GPU  Name                 Persistence-M | Bus-Id        Disp.A | Volatile Uncorr. ECC |
| Fan  Temp   Perf          Pwr:Usage/Cap |         Memory-Usage | GPU-Util  Compute M. |
|                                         |                      |               MIG M. |
|=========================================+======================+======================|
|   0  NVIDIA RTX 3500 Ada Gene...    On  | 00000000:01:00.0 Off |                  Off |
| N/A   45C    P8               4W /  35W |  10913MiB / 12282MiB |      0%      Default |
|                                         |                      |                  N/A |
+-----------------------------------------+----------------------+--

In [71]:
%pip install numba

   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 3.7/3.7 MB 3.1 MB/s eta 0:00:00a 0:00:01
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 43.9/43.9 MB 6.4 MB/s eta 0:00:0000:0100:01
Note: you may need to restart the kernel to use updated packages.


In [72]:
from numba import cuda
cuda.select_device(0)
cuda.close()

In [68]:
# Entrenar el modelo usando un ciclo de entrenamiento personalizado
epochs = 100
model.compile(optimizer=optimizador, loss=tf.keras.losses.CTC(), metrics=[calculate_bleu])
steps_per_epoch = len(train_dataset)
validation_steps = len(val_dataset)

# Iterar a través de los batches
for epoch in range(epochs):
    with tqdm(total=steps_per_epoch, desc=f'Epoch {epoch+1}/{epochs}', unit='batch') as pbar:
        epoch_loss_avg = tf.keras.metrics.Mean()
        epoch_accuracy = tf.keras.metrics.SparseCategoricalAccuracy()
    
    for epoch in tqdm(range(epochs), desc="Epoch"):

        for step, (points_batch, translation_batch) in enumerate(train_dataset):
            pbar.set_postfix({
                "Loss": epoch_loss_avg.result().numpy(),
                "Accuracy": epoch_accuracy.result().numpy()
            })
            
            with tf.GradientTape() as tape:
                predictions = model(points_batch, training=True)
                loss = tf.reduce_mean(ctc_loss(translation_batch, predictions))
                
            # Calcular y aplicar gradientes
            gradients = tape.gradient(loss, model.trainable_variables)
            model.optimizer.apply_gradients(zip(gradients, model.trainable_variables))
            predictions_decoded=decode(predictions, greedy=True)
            epoch_accuracy.update_state(translation_batch, predictions_decoded)
            epoch_loss_avg.update_state(loss)
            
            
        
        # Métricas para la validación
        val_loss_avg = tf.keras.metrics.Mean()
        val_acc_metric = tf.keras.metrics.SparseCategoricalAccuracy()
        val_bleu_avg = tf.keras.metrics.Mean()
        with tqdm(total=validation_steps, desc=f'Epoch {epoch+1}/{epochs} (Validation)', unit='batch') as pbar_val:

            for step, (points_batch, translation_batch) in enumerate(val_dataset):
                
                print("a")
                pbar_val.set_postfix({
                    "Val Loss": val_loss_avg.result().numpy(),
                    "Val Accuracy": val_acc_metric.result().numpy(),
                    "Val BLEU": val_bleu_avg.result().numpy()
                })
                
                # Evaluar en el conjunto de validación al final de cada epoch usando BLEU
                predictions = model(points_batch, training=False)

                val_loss=tf.reduce_mean(ctc_loss(translation_batch,predictions))
                val_loss_avg.update_state(val_loss)

                val_predictions = decode(predictions, greedy=False)
                val_hypotheses = [tokenizer.sequences_to_texts([pred]) for pred in val_predictions]


                val_acc_metric.update_state(translation_batch,val_predictions)
                val_references = [tokenizer.sequences_to_texts([ref.numpy()]) for ref in translation_batch]
                val_hypotheses = [tokenizer.sequences_to_texts([pred]) for pred in np.argmax(val_predictions, axis=-1)]
                val_bleu = calculate_bleu(references = val_references, hypotheses = val_hypotheses)
                val_bleu_avg.update_state(val_bleu)
            
            

        checkpoint_callback.on_epoch_end(epoch, logs={'val_loss': val_loss_avg, 'val_accuracy': val_acc_metric, 'val_bleu': val_bleu_avg})

        if early_stopping.on_epoch_end(epoch, logs={'val_loss': val_loss_avg, 'val_accuracy': val_acc_metric, 'val_bleu': val_bleu_avg}):
            print("Early stopping triggered")
            break
        print(f"Epoch: {epoch}, Loss: {epoch_loss_avg}, Accuracy: {epoch_accuracy}")
        print(f'Epoch: {epoch}, val_loss: {val_loss_avg}, val_acc: {val_acc_metric}, val_bleu: {val_bleu_avg}')

    # Evaluar el modelo en el dataset de test
    #test_loss, test_acc = model.evaluate(test_dataset)
    #print(f'Test accuracy: {test_acc}')

Epoch 1/100


2024-10-10 14:40:53.808555: E external/local_xla/xla/stream_executor/gpu/gpu_cudamallocasync_allocator.cc:310] gpu_async_0 cuMemAllocAsync failed to allocate 29687025152 bytes: CUDA error: out of memory (CUDA_ERROR_OUT_OF_MEMORY)
 Reported by CUDA: Free memory/Total memory: 0/12878086144
2024-10-10 14:40:53.808609: E external/local_xla/xla/stream_executor/gpu/gpu_cudamallocasync_allocator.cc:315] Stats: Limit:                      9809428480
InUse:                      9869051535
MaxInUse:                  10913235449
NumAllocs:                       21964
MaxAllocSize:               1625704544
Reserved:                            0
PeakReserved:                        0
LargestFreeBlock:                    0

2024-10-10 14:40:53.808733: E external/local_xla/xla/stream_executor/gpu/gpu_cudamallocasync_allocator.cc:64] Histogram of current allocation: (allocation_size_in_bytes, nb_allocation_of_that_sizes), ...;
2024-10-10 14:40:53.808739: E external/local_xla/xla/stream_executor/gpu/gp

ResourceExhaustedError: {{function_node __wrapped__Mul_device_/job:localhost/replica:0/task:0/device:GPU:0}} failed to allocate memory [Op:Mul] name: 